<div style="border: 1px solid black">
<b><center><font size="4">Ciência de Dados em Larga Escala</font></center></b>

<b><center><font size="3">Project 2</font></center></b>
</div>

**Notebook Developed by**: Rafael Antunes Lourenço<br>
**Número:** 48115 <br>
**Email:**  rafael.a.lourenco@ubi.pt<br>
<hr>

<p><a href="project1.ipynb" title="Download Notebook" download><img src="https://www.di.ubi.pt/~rcampos/assets/img_tutorials/download.jpg" align = "left" width="50" height="50" alt="Download Notebook"></a></p>
<p>&nbsp;</p>
<p>&nbsp;</p>

<hr>

# Introdução 

Este projeto aborda a análise de um extenso conjunto de dados meteorológicos diários, utilizando técnicas de chunking para processamento eficiente. Inicialmente, exploramos as variáveis disponíveis, identificando melhorias nos tipos de dados e tratando valores nulos. Em seguida, realizamos análises estatísticas, como a determinação dos anos mais antigos e recentes para cada estação, e o cálculo da temperatura média diária. Além disso, focamos nas estações em Portugal, selecionando e filtrando os dados relevantes, e substituímos os identificadores das estações pelos seus nomes correspondentes para uma melhor compreensão. Essa abordagem permite extrair insights valiosos sobre as condições climáticas ao longo do tempo e em diferentes regiões, mesmo lidando com conjuntos de dados extensos.

# Suprimir Warnings

Este pequeno excerto de código apenas serve para evitar o aparecimento das mensagens de aviso no decorrer do código, usei isto apenas porque antes de alterarmos as variáveis, havia diferentes tipos de variável em linhas iguais, o que causava muitos avisos e tornava o output cheio de avisos.

In [5]:
import warnings
import pandas as pd

# Suprimir todos os warnings
warnings.filterwarnings("ignore")

# Carregar o dataset, dividir o mesmo em chunks e guardar cada chunk no diretório /chunks

Este código lê um arquivo CSV de dados meteorológicos diários em chunks de tamanho definido (no caso, 50000) e salva cada chunk como um novo arquivo CSV. O objetivo é lidar com conjuntos de dados extensos de forma mais eficiente, dividindo-os em partes menores que podem ser processadas separadamente.


In [2]:
import pandas as pd

chunk_size=50000
batch_no=1

for chunk_df in pd.read_csv('global_climate_data/ghcnd_daily/ghcnd_daily.csv',chunksize=chunk_size):
    chunk_df.to_csv('global_climate_data/ghcnd_daily/chunks/chunks'+str(batch_no)+'.csv',index=False)
    print(batch_no)
    batch_no+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Exercicío 1 - Selecionei o chunk 200 

Escolhi o chunk 200 aleatoriamente, não existiu nenhum critério para a sua escolha. 
Defini a variável *path* com o caminho para o chunk que escolhi, para a variável *selected_columns* usei um ciclo for para selecionar todas as colunas 'id', 'year', 'month' e 'element' de todos os dias do mes, desde o value 1 ao 31.
De seguida, apenas usei a função pd.read_csv() para ler as colunas desejadas no chunk escolhido e atribui essess valores à variável *chunk200*.
Após isso, usei a função replace() para substituir todos os valores '-9999' por 'NA', usando o pd.NA.
Por final usei apenas o dtype para me indicar os tipos de variável que eu selecionei anteriormente.

In [6]:

# Caminho para o arquivo do chunk 200
path = 'global_climate_data/ghcnd_daily/chunks/chunks200.csv'

# Selecionar todas as colunas de 'value1' até 'value31'
selected_columns = ['id', 'year', 'month', 'element'] + [f'value{i}' for i in range(1, 32)]

# Leitura do chunk 200
chunk200 = pd.read_csv(path, usecols=selected_columns)

# Substituir -9999 por NaN
chunk200.replace(-9999, pd.NA, inplace=True)

# Verificar os tipos de dados
print(chunk200.dtypes)

id         object
year        int64
month       int64
element    object
value1     object
value2     object
value3     object
value4     object
value5     object
value6     object
value7     object
value8     object
value9     object
value10    object
value11    object
value12    object
value13    object
value14    object
value15    object
value16    object
value17    object
value18    object
value19    object
value20    object
value21    object
value22    object
value23    object
value24    object
value25    object
value26    object
value27    object
value28    object
value29    object
value30    object
value31    object
dtype: object


## Exercicío 1 - Converter as variáveis 

Este código converte as colunas 'value1' a 'value31' para float, 'element' para categoria, 'year' para int16 e 'month' para int8, com objetivo de otimizar o uso de memória e facilitar a manipulação dos dados.


In [7]:
# Converter as colunas 'value1' e 'value31' para float
for col in [f'value{i}' for i in range(1, 32)]:
    chunk200[col] = pd.to_numeric(chunk200[col])

# Converter a coluna 'element' para categoria
chunk200['element'] = chunk200['element'].astype('category')

# Converter a coluna 'year' para int16
chunk200['year'] = chunk200['year'].astype('int16')

# Converter a coluna 'month' para int8
chunk200['month'] = chunk200['month'].astype('int8')

# Verificar os tipos de dados atualizados
print(chunk200.dtypes)

id           object
year          int16
month          int8
element    category
value1      float64
value2      float64
value3      float64
value4      float64
value5      float64
value6      float64
value7      float64
value8      float64
value9      float64
value10     float64
value11     float64
value12     float64
value13     float64
value14     float64
value15     float64
value16     float64
value17     float64
value18     float64
value19     float64
value20     float64
value21     float64
value22     float64
value23     float64
value24     float64
value25     float64
value26     float64
value27     float64
value28     float64
value29     float64
value30     float64
value31     float64
dtype: object


# Exercicío 2 - Calcular a percentagem de valores nulos (NA)

Neste bloco de código, eu itero  sobre as colunas selecionadas do DataFrame 'chunk200'. Para cada coluna, o método 'isnull()' é aplicado, devolvendo uma série booleana indicando se cada valor é nulo ou não. Em seguida, o método 'sum()' é utilizado para somar os valores True, ou seja, contabilizo o número total de valores nulos naquela coluna. Esta contagem é então usada para calcular a percentagem de valores nulos em relação ao total de observações no DataFrame. Essa percentagem é armazenada em um dicionário, onde a chave é o nome da coluna e o valor é a percentagem de valores nulos.
 Finalmente, o loop 'for' é utilizado para imprimir cada coluna juntamente com sua respectiva percentagem de valores nulos.


In [9]:
null_percentage = {}

# Calcular a percentagem de valores nulos para cada variável
for col in selected_columns:
    # Contar o número de valores nulos na coluna
    null_count = chunk200[col].isnull().sum()
    
    # Calcular a percentagem de valores nulos
    percentage = (null_count / len(chunk200)) * 100
    
    # Armazenar a percentagem de valores nulos para a variável atual
    null_percentage[col] = percentage

for col, percentage in null_percentage.items():
    print(f'{col}: {percentage:.2f}%')

id: 0.00%
year: 0.00%
month: 0.00%
element: 0.00%
value1: 2.70%
value2: 2.47%
value3: 2.39%
value4: 2.37%
value5: 2.32%
value6: 2.23%
value7: 2.20%
value8: 2.09%
value9: 2.04%
value10: 2.13%
value11: 2.24%
value12: 2.16%
value13: 2.12%
value14: 2.12%
value15: 2.18%
value16: 2.23%
value17: 2.17%
value18: 2.20%
value19: 2.20%
value20: 2.21%
value21: 2.23%
value22: 2.22%
value23: 2.21%
value24: 2.35%
value25: 2.64%
value26: 2.43%
value27: 2.35%
value28: 2.45%
value29: 8.58%
value30: 10.52%
value31: 43.46%


# Exercicío 3 -  Determinar, para cada estação, o ano mais antigo e o mais recente.


Estblocoho de código realiza as seguintes operações:

1. **Leitura dos Arquivos**: 
   - Lê o arquivo 'ghcnd-stations.txt' para obter informações sobre as estações meteorológicas.
   - Lê o arquivo 'chunks200.csv' que contém dados meteorológicos diários.

2. **Merge de Dados**:
   - Combina os dados meteorológicos do arquivo 'chunks200.csv' com as informações das estações do arquivo 'ghcnd-stations.txt' com base no identificador da estação ('id'), utilizando a função `pd.merge()`. Os dados são combinados usando a coluna 'id' como chave primária e os dados meteorológicos são adicionados às informações da estação.

3. **Conversão de Tipos de Dados**:
   - Converte a coluna 'year' para o tipo de dados 'int16' para otimização de memória.

4. **Agrupamento de Dados**:
   - Agrupa os dados combinados por estação usando a função `groupby()`.

5. **Determinação do Ano Mais Antigo e Mais Recente**:
   - Determina o ano mais antigo e o mais recente para cada estação, encontrando o mínimo e o máximo da coluna 'year' em cada grupo de estação.

6. **Impressão dos Resultados**:
   - Imprime o resultado para cada estação, mostrando o ano mais antigo temporais e espaciais.


In [13]:
# Ler o arquivo ghcnd-stations.txt para obter informações sobre as estações
stations_info = pd.read_csv('global_climate_data/ghcnd-stations.txt', header=None, names=['id', 'name', 'latitude', 'longitude', 'elevation', 'state', 'country', 'unknown'])

# Ler o arquivo ghcnd_daily.csv
chunk200 = pd.read_csv('global_climate_data/ghcnd_daily/chunks/chunks200.csv')

# Merge entre as informações da estação e os dados meteorológicos
merged_data = pd.merge(chunk200, stations_info, on='id', how='left')

# Converter a coluna 'year' para int16
merged_data['year'] = merged_data['year'].astype('int16')

# Agrupar os dados por estação
station_groups = merged_data.groupby('id')

# Determinar o ano mais antigo e o mais recente para cada estação
oldest_year = station_groups['year'].min()
latest_year = station_groups['year'].max()

# Mostrar os resultados
for station_id, (oldest, latest) in zip(oldest_year.index, zip(oldest_year, latest_year)):
    print(f"Estação {station_id}: Ano mais antigo: {oldest}, Ano mais recente: {latest}")

Estação USC00235861: Ano mais antigo: 1938, Ano mais recente: 1968
Estação USC00235862: Ano mais antigo: 1960, Ano mais recente: 2013
Estação USC00235916: Ano mais antigo: 1985, Ano mais recente: 2019
Estação USC00235950: Ano mais antigo: 2007, Ano mais recente: 2010
Estação USC00235976: Ano mais antigo: 1893, Ano mais recente: 2019
Estação USC00235987: Ano mais antigo: 1898, Ano mais recente: 2019
Estação USC00235999: Ano mais antigo: 1893, Ano mais recente: 1894
Estação USC00236009: Ano mais antigo: 1980, Ano mais recente: 1989
Estação USC00236012: Ano mais antigo: 1956, Ano mais recente: 2019
Estação USC00236031: Ano mais antigo: 1898, Ano mais recente: 1906
Estação USC00236040: Ano mais antigo: 1893, Ano mais recente: 1901
Estação USC00236045: Ano mais antigo: 1963, Ano mais recente: 2012
Estação USC00236052: Ano mais antigo: 1898, Ano mais recente: 1910
Estação USC00236226: Ano mais antigo: 1894, Ano mais recente: 1894
Estação USC00236233: Ano mais antigo: 1893, Ano mais recente: 

# Exercicío 4 - Determinar a temperatura média para cada observação (estação/ano/mês).

1. **Seleção das Colunas**:
   - Seleciona apenas as colunas do DataFrame que começam com o prefixo 'value' usando o método `filter()` com o argumento `like='value'`.

2. **Cálculo da Temperatura Média**:
   - Calcula a temperatura média para cada observação (estação/ano/mês) através da média das colunas selecionadas anteriormente. A média é calculada ao longo do eixo das colunas (axis=1) usando o método `mean()`.

3. **Criação de uma Nova Coluna**:
   - Adiciona uma nova coluna chamada 'daily_avg_temp' ao DataFrame 'chunk200', que armazena a temperatura média calculada para cada observação.

4. **Impressão das Primeiras Linhas**:
   - Imprime as primeiras linhas do DataFrame atualizado, mostrando a nova coluna 'daily_avg_temp' juntamente com as outras colunas.

In [35]:
# Selecionar apenas as colunas que começam com 'value'
value_columns = chunk200.filter(like='value')

# Calcular a temperatura média para cada observação (estação/ano/mês)
chunk200['daily_avg_temp'] = value_columns.mean(axis=1)

# Mostrar as primeiras linhas do DataFrame com a nova coluna 'daily_avg_temp'
print(chunk200.head())

            id  year  month element value1 value2 value3 value4 value5 value6  \
0  USC00235861  1938      5    TMAX    294    267    272    244    217    217   
1  USC00235861  1938      6    TMAX    267    300    328    306    294    294   
2  USC00235861  1938      7    TMAX    339    350    367    367    367    367   
3  USC00235861  1938      8    TMAX    339    356    350    356    361    372   
4  USC00235861  1938      9    TMAX    367    322    317    328    350    339   

   ... value23 value24 value25 value26 value27 value28 value29 value30  \
0  ...     200     222     244     250     267     289     289     300   
1  ...     322     333     322     239     217     267     278     311   
2  ...     344     350     356     350     339     267     294     328   
3  ...     389     389     394     406     383     394     389     294   
4  ...     333     339     322     361     322     311     306     350   

  value31 daily_avg_temp  
0     294     245.967742  
1    <NA>     

# Exercicío 5 - Agrupar os dados por nome de estação e ano aplicando a função mean.

1. **Agrupamento de Dados**:
   - Os dados do DataFrame 'chunk200' são agrupados pelo nome da estação ('id') e pelo ano ('year') utilizando o método `groupby()`.

2. **Aplicação da Função Mean**:
   - Para cada grupo formado pela combinação única de nome da estação e ano, a função `mean()` é aplicada à coluna 'daily_avg_temp' para calcular a temperatura média.

3. **Resultados Impressos**:
   - Os resultados do cálculo da temperatura média são impressos, mostrando apenas a coluna 'daily_avg_temp' para cada combinação de estaçãoe ano.


In [36]:
# Agrupar os dados por nome da estação e ano, aplicando a função mean
station_year_avg = chunk200.groupby(['id', 'year'])['daily_avg_temp'].mean()

# Mostrar apenas os resultados para a coluna 'daily_avg_temp'
print(station_year_avg)

id           year
USC00235861  1938    259.607124
             1939     218.81918
             1940    201.160005
             1941    219.060762
             1942    206.324747
                        ...    
USC00238791  1960    190.600002
             1961    191.209082
             1962    194.580421
             1963    201.546845
             1964    203.932316
Name: daily_avg_temp, Length: 4408, dtype: object


# Exercicío 6 - Selecionar os dados apenas das estações portuguesast.

1. **Leitura do Arquivo de Informações das Estações**:
   - O arquivo 'ghcnd-stations.txt' é lido usando o método `pd.read_fwf()` para carregar os dados em um DataFrame. Esse método é utilizado devido ao formato de largura fixa do arquivo.
   - As larguras das colunas são especificadas no parâmetro 'widths'.
   - Como o arquivo não possui cabeçalho, o parâmetro 'header' é definido como None.

2. **Renomeação das Colunas**:
   - As colunas do DataFrame resultante são renomeadas para uma melhor identificação dos dados.

3. **Filtragem das Estações Meteorológicas Portuguesas**:
   - As estações meteorológicas portuguesas são filtradas com base no código do país ('id') começando com 'PO'.

4. **Seleção das Cinco Estações Desejadas**:
   - As cinco estações desejadas ('Horta', 'Funchal', 'Lisboa', 'Castelo Branco' e 'Faro') são selecionadas com base nos nomes das estações.

5. **Leitura do Arquivo de Dados Diários**:
   - O arquivo 'chunks200.csv' é lido para obter os dados meteorológicos diários.

6. **Seleção dos Dados das Estações Portuguesas**:
   - Os dados das cinco estações meteorológicas portuguesas selecionadas são extraídos do DataFrame original.

7. **Seleção das Colunas Desejadas**:
   - São selecionadas as colunas 'id', 'year', 'month', 'element' e os valores diários ('value1' a 'value31').

8. **Exibição dos Dados Selecionados**:
   - Os dados selecionados são
  
PS: Neste bloco de código o output diz ser um "EmptyDataFrama" devido ao facto de eu ter escolhido o chunk200 e ter trabalhado sempre apenas com ele, e nesse bocado do dataset não existem estações portuguesa. impressos para análise.

In [8]:
# Ler o arquivo ghcnd-stations.txt para obter informações sobre as estações
stations_info = pd.read_fwf(
    'global_climate_data/ghcnd-stations.txt',
    widths=[11, 9, 10, 7, 31, 3, 3, 3, 5],
    header=None
)

# Renomear as colunas para melhor identificação
stations_info.columns = ['id', 'lat', 'long', 'elev', 'name', 'state', 'GSN_FLAG', 'HCN/CRN_FLAG', 'WMO_ID']

# Filtrar as estações meteorológicas portuguesas
portuguese_stations = stations_info[stations_info['id'].str[:2] == 'PO']

# Selecionar as cinco estações desejadas
portuguese_station_names = ['Horta', 'Funchal', 'Lisboa', 'Castelo Branco', 'Faro']
selected_stations = portuguese_stations[portuguese_stations['name'].isin(portuguese_station_names)]

# Ler o arquivo ghcnd_daily.csv
chunk200 = pd.read_csv('global_climate_data/ghcnd_daily/chunks/chunks200.csv')

# Selecionar os dados das cinco estações meteorológicas portuguesas
selected_data = chunk200[chunk200['id'].isin(selected_stations['id'])]

# Selecionar as colunas desejadas
selected_columns = ['id', 'year', 'month', 'element'] + [f'value{i}' for i in range(1, 32)]
selected_data = selected_data[selected_columns]

# Mostrar os dados selecionados
print(selected_data)

Empty DataFrame
Columns: [id, year, month, element, value1, value2, value3, value4, value5, value6, value7, value8, value9, value10, value11, value12, value13, value14, value15, value16, value17, value18, value19, value20, value21, value22, value23, value24, value25, value26, value27, value28, value29, value30, value31]
Index: []

[0 rows x 35 columns]


# Exercicío 7 - Substituir os ids presentes no dataframe pelo correspondente nome da estação.

1. **Leitura do Arquivo de Informações das Estações**:
   - O arquivo 'ghcnd-stations.txt' é lido usando o método `pd.read_fwf()` para carregar os dados em um DataFrame. As larguras das colunas são especificadas no parâmetro 'widths'.
   - As colunas do DataFrame resultante são renomeadas para uma melhor identificação dos dados.

2. **Leitura do Arquivo de Dados Diários**:
   - O arquivo 'chunks200.csv' é lido para obter os dados meteorológicos diários.

3. **Merge entre as Informações da Estação e os Dados Meteorológicos**:
   - Os dados do arquivo 'chunks200.csv' são mesclados com as informações das estações com base no ID da estação.
   - A mesclagem é feita usando o método `pd.merge()`.

4. **Substituição do ID pelo Nome da Estação**:
   - A coluna 'id' é removida do DataFrame resultante.
   - A coluna 'name', que contém o nome da estação, é renomeada para 'station_name'.

5. **Exibição dos Dados com os Nomes das Estações**:
   - Os dados resultantes, agora com os IDs substituídos pelo nome da estação, são impresss para análise.


In [40]:
# Ler o arquivo ghcnd-stations.txt para obter informações sobre as estações
stations_info = pd.read_fwf('global_climate_data/ghcnd-stations.txt', 
                            widths=[11, 8, 9, 6, 2, 30, 3, 3, 5],
                            header=None,
                            names=['id', 'latitude', 'longitude', 'elevation', 'state', 'name', 'GSN_FLAG', 'HCN/CRN_FLAG', 'WMO_ID'])

# Ler o arquivo ghcnd_daily.csv
chunk200 = pd.read_csv('global_climate_data/ghcnd_daily/chunks/chunks200.csv')

# Merge entre as informações da estação e os dados meteorológicos
merged_data = pd.merge(chunk200, stations_info[['id', 'name']], on='id', how='left')

# Substituir a coluna 'id' pelo nome correspondente da estação
merged_data.drop(columns=['id'], inplace=True)
merged_data.rename(columns={'name': 'station_name'}, inplace=True)

# Mostrar os dados com os IDs substituídos pelo nome da estação
print(merged_data)

       year  month element  value1  mflag1 qflag1 sflag1  value2  mflag2  \
0      1938      5    TMAX     294     NaN    NaN      0     267     NaN   
1      1938      6    TMAX     267     NaN    NaN      0     300     NaN   
2      1938      7    TMAX     339     NaN    NaN      0     350     NaN   
3      1938      8    TMAX     339     NaN    NaN      0     356     NaN   
4      1938      9    TMAX     367     NaN    NaN      0     322     NaN   
...     ...    ...     ...     ...     ...    ...    ...     ...     ...   
49995  1964      3    TMAX     189     NaN    NaN      0     183     NaN   
49996  1964      4    TMAX     150     NaN    NaN      0     239     NaN   
49997  1964      5    TMAX     256     NaN    NaN      0     272     NaN   
49998  1964      6    TMAX     250     NaN    NaN      0     228     NaN   
49999  1964      7    TMAX     300     NaN    NaN      0     333     NaN   

      qflag2  ... sflag29  value30  mflag30 qflag30 sflag30  value31  mflag31  \
0     

# Conclusão

Ao longo deste trabalho, foi possível consolidar e aplicar diversos conhecimentos relacionados ao processamento de grandes conjuntos de dados, especialmente utilizando a técnica de chunking para lidar com arquivos de tamanho considerável. Através da análise e manipulação desses dados meteorológicos, foram aplicadas técnicas como leitura de arquivos CSV e TXT, tratamento de dados ausentes, conversão de tipos de dados, fusão de DataFrames, seleção e filtragem de dados, entre outras.

Além disso, foi possível praticar habilidades relacionadas ao trabalho com DataFrames no Pandas, como renomear colunas, calcular estatísticas descritivas, agrupar dados e aplicar funções de agregação.

É importante destacar que, ao longo do processo, recorri ao ChatGPT para esclarecer dúvidas sobre determinadas partes do código, corrigir pequenos erros e ajudar-me a documentar de maneira completa e com maior organização o meu trabalho